In [1]:
import os
import angr



proj = angr.Project('/home/nahid/reverse/binaries/a.out')

WARNING  | 2023-02-07 13:46:21,443 | cle.loader     | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [2]:
state = proj.factory.entry_state()

In [3]:
# 64-bit bitvectors with concrete values 1 and 100
one = state.solver.BVV(1, 64)
print(one)

 
one_hundred = state.solver.BVV(100, 64)
print(one_hundred)
 

# create a 27-bit bitvector with concrete value 9
weird_nine = state.solver.BVV(9, 27)
print(weird_nine)

<BV64 0x1>
<BV64 0x64>
<BV27 0x9>


In [4]:


byteStr = b"\x55\x48\x8b\x05\xb8\x13\x00\x00" 
print(len(byteStr))
temp = state.solver.BVV(byteStr,64)
print(temp)
print(one+ temp)



onezero = state.solver.BVV(12297829382473034410,64) 
print(onezero)
print(onezero/2)

8
<BV64 0x55488b05b8130000>
<BV64 0x55488b05b8130001>
<BV64 0xaaaaaaaaaaaaaaaa>
<BV64 0x5555555555555555>


In [5]:
print(one + one_hundred)


# You can provide normal Python integers and they will be coerced to the appropriate type:
print(one_hundred + 0x100)


# The semantics of normal wrapping arithmetic apply
print(one_hundred - one*200)


print(weird_nine)
weird_nine.zero_extend(64 - 27)
print(weird_nine)

print(one + weird_nine.zero_extend(64 - 27))



<BV64 0x65>
<BV64 0x164>
<BV64 0xffffffffffffff9c>
<BV27 0x9>
<BV27 0x9>
<BV64 0xa>


In [6]:
# Create a bitvector symbol named "x" of length 64 bits
x = state.solver.BVS("x", 64)
print(x)

y = state.solver.BVS("y", 64)
print(y)


<BV64 x_49_64>
<BV64 y_50_64>


In [7]:
print(x + one)


print((x + one) / 2)


print( x - y)


<BV64 x_49_64 + 0x1>
<BV64 (x_49_64 + 0x1) / 0x2>
<BV64 x_49_64 - y_50_64>


In [7]:
tree = (x + 1) / (y + 2)
print(tree)

print(tree.op)

print(tree.args)


print(tree.args[0].op)

print(tree.args[0].args)


print(tree.args[0].args[1].op)


print(tree.args[0].args[1].args)


<BV64 (x_49_64 + 0x1) / (y_50_64 + 0x2)>
__floordiv__
(<BV64 x_49_64 + 0x1>, <BV64 y_50_64 + 0x2>)
__add__
(<BV64 x_49_64>, <BV64 0x1>)
BVV
(1, 64)


In [9]:
# get a fresh state without constraints
state = proj.factory.entry_state()

input = state.solver.BVS('input', 64)

operation = input * 5 + 100
output = 200
state.solver.add(operation < output)
state.solver.eval(input)


0

In [10]:
state = proj.factory.entry_state()
input = state.solver.BVS('input', 64)
operation = input <100
output = 200

state.solver.add(operation == output)
state.solver.satisfiable()

False